In [1]:
!pip install yfinance

In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

## Data

In [34]:
date_1, date_2 = "2024-01-01", "2024-08-31"

# Downlaod Data
ser_1 = yf.download('HAL', start=date_1, end=date_2)[['Adj Close']]
ser_2 = yf.download('RELIANCE.NS', start=date_1, end=date_2)[['Adj Close']]
ser_3 = yf.download('TATAMOTORS.NS', start=date_1, end=date_2)[['Adj Close']]

idx_1 = yf.download('^NSEI', start=date_1, end=date_2)[['Adj Close']]
idx_2 = yf.download('^BSESN', start=date_1, end=date_2)[['Adj Close']]

# Rename Columns
ser_1.columns = ['HAL']
ser_2.columns = ['RELIANCE']
ser_3.columns = ['TATAMOTORS']
idx_1.columns = ['NSEI']
idx_2.columns = ['BSESN']

# Join all data together
df = ser_1.join(ser_2).join(ser_3).join(idx_1).join(idx_2)

df.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,HAL,RELIANCE,TATAMOTORS,NSEI,BSESN
Date,,,,,
2024-01-02,35.561352,2602.865967,781.986877,21665.800781,71892.476562
2024-01-03,36.004395,2574.562012,779.045959,21517.349609,71356.601562
2024-01-04,35.226612,2587.866699,793.301941,21658.599609,71847.570312
2024-01-05,35.413673,2598.879395,788.516724,21710.800781,72026.148438
2024-01-08,34.665428,2578.598389,786.672363,21513.000000,71355.218750


In [52]:
# Checking Nulls

nulls = df[df.isnull().any(axis=1)]
nulls

,HAL,RELIANCE,TATAMOTORS,NSEI,BSESN
Date,,,,,
2024-01-22,33.897491,NaN,NaN,NaN,NaN
2024-01-26,36.949543,NaN,NaN,NaN,NaN
2024-03-08,35.823067,NaN,NaN,NaN,NaN
2024-03-25,38.405178,NaN,NaN,NaN,NaN
2024-04-11,40.502525,NaN,NaN,NaN,NaN
2024-04-17,37.633514,NaN,NaN,NaN,NaN
2024-05-01,35.941788,NaN,NaN,NaN,NaN
2024-05-20,37.554371,NaN,NaN,NaN,NaN
2024-06-17,33.219723,NaN,NaN,NaN,NaN


In [65]:
# Getting Numerical Index from Dataframe
null_idx = [df.index.get_loc(x) for x in sorted(nulls.index)]

# Filling Nulls With Average of Previous and Next Availabel Value
for idx in nulls.index:
    df.iloc[idx, 1:] = (df.iloc[idx-1, 1:] + df.iloc[idx+1, 1:])/2

In [70]:
# Normalize Data [Scale Down]
norm_factor = df.iloc[0]
df = df/df.iloc[0]

df.head()

,HAL,RELIANCE,TATAMOTORS,NSEI,BSESN
Date,,,,,
2024-01-02,1.000000,1.000000,1.000000,1.000000,1.000000
2024-01-03,1.012459,0.989126,0.996239,0.993148,0.992546
2024-01-04,0.990587,0.994237,1.014470,0.999668,0.999375
2024-01-05,0.995847,0.998468,1.008350,1.002077,1.001859
2024-01-08,0.974806,0.990677,1.005992,0.992947,0.992527


In [74]:
df.shift(-1)

,HAL,RELIANCE,TATAMOTORS,NSEI,BSESN
Date,,,,,
2024-01-02,1.012459,0.989126,0.996239,0.993148,0.992546
2024-01-03,0.990587,0.994237,1.014470,0.999668,0.999375
2024-01-04,0.995847,0.998468,1.008350,1.002077,1.001859
2024-01-05,0.974806,0.990677,1.005992,0.992947,0.992527
2024-01-08,0.955426,0.988054,1.019633,0.994417,0.992958
...,...,...,...,...,...
2024-08-26,0.884104,1.152921,1.377581,1.154712,1.136583
2024-08-27,0.866768,1.151269,1.374128,1.156309,1.137609
2024-08-28,0.877393,1.168654,1.434359,1.160906,1.142465
